In [ ]:
import glob
from catboost.utils import get_gpu_device_count
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
is_gpu_available = get_gpu_device_count()
device = 'GPU' if is_gpu_available else 'CPU'

device

Error 35 (CUDA driver version is insufficient for CUDA runtime version) ignored while obtaining device count


'CPU'

In [ ]:
TRAIN_PATH = r'/data/Кейс-3. Отток юридических лиц из расчетно-кассового обслуживания/train'
TEST_PATH = r'/data/Кейс-3. Отток юридических лиц из расчетно-кассового обслуживания/test'

In [4]:
def get_data(path):
    filenames_train = glob.glob(path + '/*.csv')
    data_files_train = []
    
    for filename in filenames_train:
        data_files_train.append(pd.read_csv(filename))

    return pd.concat(data_files_train, ignore_index=True)

train_df = get_data(TRAIN_PATH)
test_df = get_data(TEST_PATH)

In [5]:
ids = test_df['id'].tolist()

In [ ]:
f = [
       'feature_168', 'feature_87', 'feature_72', 'feature_124', 'feature_141',
       'feature_29', 'feature_55', 'feature_142', 'feature_78', 'feature_183',
       'feature_84', 'feature_146', 'feature_134', 'feature_26', 'feature_12',
       'feature_127', 'feature_59', 'feature_100', 'feature_96', 'feature_112',
       'feature_169', 'feature_16', 'feature_76', 'feature_81', 'feature_79',
       'feature_22', 'feature_152', 'feature_43', 'feature_20', 'feature_18',
       'feature_44', 'id', 'feature_177', 'feature_50', 'feature_6',
       'feature_66', 'feature_9', 'feature_46', 'feature_103', 'feature_75',
       'feature_8', 'feature_36', 'feature_41', 'feature_184', 'feature_62',
       'feature_95', 'feature_133', 'feature_28', 'feature_108', 'feature_128',
       'feature_117', 'feature_161', 'feature_157', 'feature_107', 'feature_147'
]

In [ ]:
train_df = train_df[f + ['target']]
test_df = test_df[f]

In [8]:
feature_operations = {
    'atan_feature_112': ('feature_112', np.arctan),
    'log_feature_26': ('feature_26', np.log1p),
    'sin_feature_26': ('feature_26', np.sin),
    'tan_feature_26': ('feature_26', np.tan)
}

for new_feature, (base_feature, operation) in feature_operations.items():
    train_df[new_feature] = operation(train_df[base_feature])
    test_df[new_feature] = operation(test_df[base_feature])

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
!pip install -U lightautoml

  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.4/416.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 34.7 MB/s eta 0:

In [10]:
import random
import torch
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import os

In [11]:
N_THREADS = 4
N_FOLDS = 7
RANDOM_STATE = 52
TIMEOUT = 36000
TARGET_NAME = 'target'

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(RANDOM_STATE)

In [13]:
torch.set_num_threads(N_THREADS)

In [14]:
# automl = TabularAutoML(
#     task=Task('binary', loss='logloss', metric='auc'), 
#     reader_params={'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': False},
#     timeout=TIMEOUT,
#     cpu_limit=N_THREADS,
#     debug=True,
#     general_params={
#         "use_algos": [
#             [
#                 'lgb', 
#                 'lgb_tuned', 
#                 'xgb', 
#                 'xgb_tuned', 
#                 'fttransformer_tuned', 
#                 'autoint_tuned'
#             ]
#         ]
#     },
#     nn_params={
#         "0": {
#             "bs": 1024,
#             'lr': 1e-3,
#             "freeze_defaults": True,
#             "cont_embedder": 'plr',
#             "n_epochs": 15,
#             'path_to_save': '/kaggle/working/fttransformer',
#         },
#         '1': {
#             "bs": 1024,
#             'lr': 1e-3,
#             "freeze_defaults": True,
#             "n_epochs": 15,
#             'path_to_save': '/kaggle/working/autoint',
#         }
#     },
#     tuning_params={'max_tuning_time': 3600, 'max_tuning_iter': 101, 'fit_on_holdout': True},
#     selection_params={'mode': 0},
#     gpu_ids='0'
# )

automl = TabularAutoML(
    task=Task('binary', loss='logloss', metric='auc'), 
    reader_params={'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE, 'advanced_roles': False},
    tuning_params={'max_tuning_time': 3600, 'max_tuning_iter': 200, 'fit_on_holdout': True},
    timeout=TIMEOUT,
    cpu_limit=N_THREADS
)

In [15]:
%%time
oof_pred = automl.fit_predict(train_df, roles={'target': TARGET_NAME}, verbose=3)

[10:41:12] Stdout logging level is INFO3.
[10:41:12] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[10:41:12] Task: binary

[10:41:12] Start automl preset with listed constraints:
[10:41:12] - time: 36000.00 seconds
[10:41:12] - CPU: 4 cores
[10:41:12] - memory: 16 GB

[10:41:12] Train data shape: (413194, 60)

[10:41:13] Layer 1 train process start. Time left 35998.76 secs
[10:41:15] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[10:41:15] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[10:41:15] Linear model: C = 1e-05 score = 0.7797954896547298
[10:41:16] Linear model: C = 5e-05 score = 0.7826892712398302
[10:41:16] Linear model: C = 0.0001 score = 0.7834345714976454
[10:41:16] Linear model: C = 0.0005 score = 0.7840509723841159
[10:41:16] Linear model: C = 0.001 score = 0.7840509723841159
[10:41:16] Linear model: C = 0.005 score = 0.7840507579307839
[10:41:16] ===== Start working with fold 1 for Lvl_0_Pipe_

In [16]:
print(automl.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 0.17114 * (7 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.65639 * (7 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.11405 * (7 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.05842 * (7 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) 


In [17]:
print(f'TRAIN out-of-fold score: {roc_auc_score(train_df[TARGET_NAME].values, oof_pred.data.flatten())}')

TRAIN out-of-fold score: 0.8133024430267657


In [18]:
%%time
pred = automl.predict(test_df).data.flatten()

CPU times: user 1min 4s, sys: 84.9 ms, total: 1min 4s
Wall time: 18 s


In [ ]:
import csv

subm = pd.read_csv('/data/Кейс-3. Отток юридических лиц из расчетно-кассового обслуживания/baseline_submission_case3.csv')
subm['target'] = pred
subm['id'] = ids
subm['id'] = subm['id'].astype(int)

with open('submission_laml.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(subm.columns)
    writer.writerows(subm.values)